### 볼린저 밴드 
1. 파일 로드 
2. 결측치와 이상치를 확인 및 제거 
3. 시간과 종가만의 데이터프레임으로 변경
4. 이동 평균선 : 데이터 20개의 평균을 구해서 새로운 파생변수에 대입
5. 상단 밴드 : 이동 평균선 + (2 * 데이터 20개의 표준편차)
6. 하단 밴드 : 이동 평균선 - (2 * 데이터 20개의 표준편차)
7. 구매 상태를 확인하는 파생변수 생성
8. 구매 상태를 입력
9. 수익율 계산

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("../csv/BND.csv", index_col='Date')
df.head()

In [ ]:
## 결측치와 이상치를 확인
df[df.isin([np.nan, np.inf, -np.inf]).any(1)]

In [ ]:
## 결측치와 이상치를 제외한 데이터 생성
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]

In [ ]:
## 시간과 종가만으로 이루어져있는 데이터프레임 생성

# price_df = df.loc[ : , ["Adj Close"]]

price_df = df[["Adj Close"]]

In [ ]:

def bol(x):

    # if i >= 19:
    #     result = price_df.iloc[i-19:i].mean()
    #     return result
    # print(x , end = " ")

price_df.apply(bol, axis=1, raw=1)

In [ ]:
## 이동 평균선
# 데이터양이 20개의 평균을 구해서 새로운 파생변수 대입
# 과거의 데이터의 수가 20개가 되지 않으면 결측치로 출력

price_df["center"] = price_df['Adj Close'].rolling(20).mean()

In [ ]:
price_df.iloc[18:25]

In [ ]:
## ub 파생변수를 생성 
## 이동평균선 + (2 * 종가 데이터 20개의 표준편차)
price_df["ub"] = \
    price_df["center"] + ( 2 * price_df["Adj Close"].rolling(20).std() )

In [ ]:
## lb 파생변수 생성
# 이동평균선 - ( 2 * 종가 데이터 20개의 표준편차 )
price_df["lb"] = \
    price_df['center'] - ( 2 * price_df['Adj Close'].rolling(20).std() )

In [ ]:
price_df.iloc[18:25]

In [ ]:
price_df.plot()

In [ ]:
## 시작 시간 생성 
start_time = '2010-01-02'
price_df_2 = price_df.loc[start_time:]
price_df_2.head(3)

In [ ]:
## 거래 내역이라는 파생변수 생성
## 비어있는 값을 생성
price_df_2['trade'] = ""
price_df_2.head(2)


### 구매내역 추가
- 조건 
    1. 상단 밴드보다 종가가 높은 경우
        - 현재 구매 상태인 경우
            - 매도 trade = ""
        - 현재 구매 상태가 아닌 경우
            - 아무 행동도 하지 않는다. trade = ""
    2. 하단 밴드보다 종가가 낮은 경우
        - 현재 구매 상태인 경우
            - 구매 상태를 유지 trade = "buy"
        - 현재 구매 상태가 아닌 경우
            - 구매 trade = "buy"
    3. 하단 밴드와 상단 밴드 사이에 종가가 존재하면
        - 현재 구매 상태인 경우
            - 구매 상태를 유지 trade = "buy"
        - 현재 구매 상태가 아닌 경우
            - 현 상태를 유지 trade = ""

In [ ]:
for i in price_df_2.index:
    if price_df_2.loc[i, 'Adj Close'] > price_df_2.loc[i, 'ub']:
        if price_df_2.shift(1).loc[i, 'trade'] == "buy":
            price_df_2.loc[i, 'trade'] = ""
        else : 
            price_df_2.loc[i, 'trade'] = ""
    elif price_df_2.loc[i, 'Adj Close'] < price_df_2.loc[i, 'lb']:
        if price_df_2.shift(1).loc[i, 'trade'] == "buy":
            price_df_2.loc[i, 'trade'] = 'buy'
        else : 
            price_df_2.loc[i, 'trade'] = 'buy'
    elif price_df_2.loc[i, 'Adj Close'] >= price_df_2.loc[i, 'lb'] and \
        price_df_2.loc[i, 'Adj Close'] <= price_df_2.loc[i, 'ub']:
        if price_df_2.shift(1).loc[i, 'trade'] == 'buy':
            price_df_2.loc[i, 'trade'] = 'buy'
        else:
            price_df_2.loc[i, 'trade'] = ""


In [ ]:
price_df_2['trade'].value_counts()

In [ ]:
# price_df_2[price_df_2.isin(["buy"]).any(1)]
# price_df_2.query("trade == 'buy'")
price_df_2[price_df_2['trade'] == 'buy']

In [74]:
## apply를 이용하여 구매 내역 추가

def bol(x):
    close = x[0]
    ub = x[1]
    lb = x[2]
    trade = price_df_2.shift(1)[(price_df_2["Adj Close"] == close) & 
                    (price_df_2["ub"] == ub) & 
                    (price_df_2["lb"] == lb)]['trade'].values
    if close > ub:
        return ""
    elif close < lb:
        return 'buy'
    else:
        if trade == 'buy':
            return 'buy'
        else : 
            return ""


price_df_2[["Adj Close", 'ub', 'lb']].apply(bol, axis=1, raw=1).value_counts()

       1568
buy     816
dtype: int64

In [ ]:
def func(x):
    close = x[0]
    ub = x[1]
    lb = x[2]
    if price_df_2.shift(1)[(price_df_2["Adj Close"] == close) & (price_df_2["ub"] == ub) & (price_df_2['lb'] == lb)]['trade'] == "":
        print("None")

price_df_2[["Adj Close", 'ub', 'lb']].apply(func, axis=1, raw=1)

### 수익율 계산
- 구매를 한 날의 종가
    - trade 컬럼에서 전 행의 trade가 "" 현재 행 trade가 "buy"인 날의 종가 = 구매 가격
- 판매를 한 날의 종가
    - 전 행의 trade가 "buy"이고 현재 행 trade가 "" 인 날의 종가 = 판매 가격
- 수익율 계산
    - (판매 가격 - 구매 가격) / 구매 가격 + 1
- 구매 가격과 판매 가격를 초기화
- 여러개의 수익율 발생
- 누적 수익율
    - 수익율 누적으로 곱

In [ ]:
# 손익 계산
rtn = 1.0
price_df_2["return"] = 1
buy = 0.0
sell = 0.0
for i in price_df_2.index:
    ## 구매가를 출력
    if price_df_2.shift(1).loc[i, "trade"] == '' and \
        price_df_2.loc[i, 'trade'] == 'buy':
        buy = price_df_2.loc[i, 'Adj Close']
        print('진입일 :', i, "구매 가격 :", buy)
    ## 판매가를 출력
    elif price_df_2.shift(1).loc[i, 'trade'] == 'buy' and \
        price_df_2.loc[i, 'trade'] == '':
        sell = price_df_2.loc[i, 'Adj Close']
        rtn = (sell - buy) / buy + 1
        price_df_2.loc[i, 'return'] = rtn
        print('청산일 :', i, "판매 가격 :", sell, "수익율 :", round(rtn, 4))
    
    ## buy, sell 변수를 초기화
    if price_df_2.loc[i, 'trade'] == "":
        buy = 0.0
        sell = 0.0
    
    


## for문을 이용하여 구매 가격 print, 판매 가격 print()
# 구매인 경우 조건 : 전 행의 trade가 "" 이고(그리고) 현재 행의 trade가 "buy"
# 판매인 경우 조건 : 전 행의 trade가 "buy" 이고(그리고) 현재 행의 trade가 ""
# 이 두 경우에 종가를 print()

In [79]:
##누적 수익율

acc_rtn = 1.0

for i in price_df_2.index:
    rtn = price_df_2.loc[i, 'return']
    acc_rtn *= rtn
    price_df_2['acc_rtn'] = acc_rtn

print("누적 수익율 : ", round(acc_rtn, 4))

C:\Users\moons\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


누적 수익율 :  1.0794


### 볼린저 밴드를 함수화
- 1번 함수 생성
    - 매개변수 데이터프레임, 기준이되는 컬럼명, 시작 시간 생성
    - 데이터프레임의 결측치와 이상치를 제거
    - 기준되는 컬럼만 두고 나머지 컬럼은 삭제
    - 이동 평균선, 상단 밴드, 하단 밴드 파생 변수 생성
    - 시작 시간부터 마지막 시간까지 데이터프레임을 필터링 한 데이터프레임을 리턴
- 2번 함수 생성
    - 매개변수 데이터프레임 생성
    - trade 컬럼을 생성 -> 값 ""
    - 구매내역을 trade 컬럼 대입 후 데이터프레임 리턴
- 3번 함수 생성
    - 매개변수 데이터프레임 생성
    - return 컬럼을 생성 -> 값 1
    - 손익 계산을 하여 return 컬럼에 대입 
    - 누적 수익율 acc_rtn 파생변수를 생성 누적 수익율 대입
    - 누적 수익율 리턴

- 위의 내용이 잘 안되는 분들은 
- 함수를 생성 
- 매개변수 3개 (데이터프레임, 시작 시간, 컬럼명)
- 데이터프레임에서 해당하는 컬럼명만 있는 데이터프레임 생성 (loc, [[]])
- 시작 시간부터 마지막 데이터까지(loc[행의 조건, 열의 조건]) 출력이되는 데이터프레임을 리턴

In [83]:
def test(data, start, col):
    result = data.loc[start : , [col]]
    return result


df = pd.read_csv("../csv/AAPL.csv", index_col='Date')

test(df, '2010-01-01', 'Adj Close')

,Adj Close
Date,
2010-01-04,26.782711
2010-01-05,26.829010
2010-01-06,26.402260
2010-01-07,26.353460
2010-01-08,26.528664
...,...
2019-06-18,198.449997
2019-06-19,197.869995
2019-06-20,199.460007


In [84]:
### 1번 함수 
def first(data, col, start):
    ## data : 데이터프레임, col : 컬럼의 이름, start : 시간

    ## 결측치 , 이상치를 제거
    result = data[~data.isin([np.nan, np.inf, -np.inf]).any(1)]
    ## 해당 컬럼만 남기고 나머지 컬럼은 삭제
    result = result.loc[:, [col]]
    ## 이동 평균선, 상단 밴드, 하단 밴드 생성
    result['center'] = result[col].rolling(20).mean()
    result['ub'] = result['center'] + ( 2 * result[col].rolling(20).std() )
    result['lb'] = result['center'] - ( 2 * result[col].rolling(20).std() )

    result = result.loc[start :]

    return result


In [85]:
df = pd.read_csv("../csv/AAPL.csv", index_col='Date')

In [86]:
first(df, 'Close' ,'2010-01-01')

,Close,center,ub,lb
Date,,,,
2010-01-04,30.572857,28.580929,30.874244,26.287614
2010-01-05,30.625713,28.731357,31.149766,26.312948
2010-01-06,30.138571,28.888643,31.239205,26.538081
2010-01-07,30.082857,29.036571,31.289988,26.783154
2010-01-08,30.282858,29.137857,31.425605,26.850109
...,...,...,...,...
2019-06-18,198.449997,185.432500,201.032574,169.832427
2019-06-19,197.869995,185.996000,202.558154,169.433846
2019-06-20,199.460007,186.830000,204.361771,169.298229
